University of Helsinki, Master's Programme in Mathematics and Statistics  
MAST32001 Computational Statistics, Autumn 2023  
Luigi Acerbi  

# Week 5 exercises

For this week exercises, you can implement your code using either Autograd or PyTorch as you prefer.

Autograd provides a simpler, more NumPy-like interface, while PyTorch provides a more powerful framework for advanced users. If in doubt, it is probably better to try Autograd. You can port your code to PyTorch later if you wish to experiment with that.

Please see Chapter 10 of the course notes and this week's computer tasks for more details and examples of use of both frameworks.

-----------------------------------

*Hints*:

- If you are using Autograd, make sure you always import `autograd.numpy` instead of regular `numpy` and `autograd.scipy` instead of regular `scipy` to allow Autograd to differentiate any functions you use. This can usually be achieved easily using
  ```
  import autograd.numpy as np
  import autograd.scipy.special as scs
  ```
  and using the packages via `np` and `scs` aliases as usual.
- The import trick above is not needed for PyTorch, which does not support differentiation of NumPy and SciPy functions, only its own functions.
- If you are having problems with Autograd, please note the following:
  * Autograd does not support subscripted assignment (e.g. `a[1] = 2`) and will give a mysterious error if you try.
  * Autograd does not support named function arguments (e.g. `f(foo=1, bar=2)`) and will give a mysterious error if you try.
  * Autograd can only differentiate functions in `autograd.numpy` and `autograd.scipy`. Check your imports if you get mysterious errors.
- If you use PyTorch, please make sure to use the `double` data type. The accuracy of the default data type `float` may not be sufficient.

In [1]:
import torch
from torch import Tensor, tensor
from torch import Tensor
from typing import Callable
from functools import partial
# PyTorch code
import math

import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt

from dataclasses import dataclass

plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi'] = 100
sns.set_style("darkgrid")

torch.set_default_dtype(torch.double)
if torch.cuda.is_available():
    torch.set_default_tensor_type(torch.cuda.DoubleTensor)


## 1. Gradients of a multivariate normal likelihood (4 points)

We will test the evaluation and gradient of the multivariate normal likelihood using a covariance matrix specified by a Gaussian process. A 1D Gaussian process is a distribution over continuous functions such that the joint distribution over any finite set of points is multivariate normal (Gaussian). We will use the so-called *squared exponential* covariance function over input points $t_1, \dots, t_n$ defined by  
$$ k(t_i, t_j; \ell) = \exp\left( -\frac{(t_i-t_j)^2}{2 \ell^2} \right), $$
where the parameter $\ell$ is called the *length scale* of the process.

1. Write a function to generate a $3 \times 3$ covariance matrix $K$ such that $K_{i,j} = k(t_i, t_j; \ell)$ where $t_1 = 1, t_2 = 2, t_3 = 3$ as a function of $\ell$.
2. Evaluate the log-pdf of the vector $x = (1, 0, 1)$ under a multivariate normal with $\textbf{0}$ mean and the above covariance matrix $K$ with $\ell = 2$. Report the value of the log-pdf to Moodle.
3. Evaluate the finite difference approximation to the gradient of the log-pdf with respect to $\ell$ at the point $\ell=2, x=(1, 0, 1)$ with $h = 10^{-3}$ (`1e-3`) and report it to Moodle. *Note*: Use the first-order finite difference formula presented in Section 10.2 of the lecture notes
4. Evaluate the exact gradient of the log-pdf with respect to $\ell$ at the point $\ell = 2, x = (1, 0, 1)$ using automatic differentiation and report it to Moodle.

*Hint*: you can generate a matrix of differences of $t$ values in NumPy using the `None` index to create a new dimension:
```
t = np.array([1.0, 2.0, 3.0])
dt = t[:,None] - t[None,:]
```

In PyTorch, the corresponding trick works but the input needs to be a `tensor`:
```
torch.set_default_dtype(torch.double)
# Uncomment this to run on GPU:
# torch.set_default_tensor_type(torch.cuda.DoubleTensor)

t = torch.tensor([1.0, 2.0, 3.0])
dt = t[:,None] - t[None,:]
```

Useful PyTorch functions for evaluating the multivariate normal density that differ from NumPy: `torch.linalg.solve()`, `torch.dot()`

In [ ]:


def k(t: Tensor, l: Tensor) -> Tensor:
    breakpoint()
    return torch.exp(- (t[:, None] - t[None, :]).pow(2) / (2*l.pow(2)))

covar = k(tensor([1, 2, 3]), tensor([2]))

def multivariate_normal_log_pdf(loc: Tensor, k: Callable[[Tensor, Tensor], Tensor], t: Tensor, x: Tensor, l: Tensor) -> Tensor:
    return torch.distributions.MultivariateNormal(loc, k(t, l)).log_prob(x)


multivariate_normal_log_pdf(loc=torch.zeros(3), k=k, t=tensor([1,2,3]), x=tensor([1., 0., 1.]), l=tensor(2))

In [ ]:
l = tensor(2.0, requires_grad=True)

f_x = partial(multivariate_normal_log_pdf, loc=torch.zeros(3), k=k, t=tensor([1, 2, 3]), x=tensor([1., 0., 1.]))

f_x(l=l).backward()
print(l.grad)
del(l)


In [ ]:
l = tensor(2.0)
print((f_x(l=l+1e-3) - f_x(l=l)) / 1e-3)
del(l)

## 2. Maximum likelihood estimation of the gamma distribution (4 points)

In this exercise we test maximum likelihood estimation of the [Gamma distribution](https://en.wikipedia.org/wiki/Gamma_distribution) to a given data set.

Note that the parameters $\alpha$ and $\beta$ of the Gamma distribution need to be positive. This can be accomplised by using $a$ and $b$ satisfying $\alpha = \exp(a)$ and $\beta = \exp(b)$ as variables in the optimization.

1. Load the data set $\mathcal{D} = \{ x_i | i = 1, \dots, n \}$ using the function below.
2. Find the maximum likelihood fit for the parameters $\alpha$ and $\beta$ of the Gamma distribution $\Gamma(\alpha, \beta)$ when $x_i \sim \Gamma(\alpha, \beta)$.
3. Plot the maximum likelihood distribution together with a normed histogram of the data to check your estimates.
4. Report your estimates $\alpha$ and $\beta$ in Moodle. (*Note*: please remember to report $\alpha$ and $\beta$, not $a$ and $b$.)

*Hints*: 
- Feel free to use the implementation of the Gamma distribution pdf provided below. Note that here we are using the shape-rate parametrization of the Gamma pdf.
- Note that we are performing maximum-likelihood estimation, not Bayesian inference. There is no prior here. So, there is no prior density that needs to be corrected when you transform from $\alpha \rightarrow a$ and $\beta \rightarrow b$.

In [ ]:


data = tensor(pd.read_csv('https://raw.githubusercontent.com/lacerbi/compstats-files/main/data/toydata.txt', sep='\t', header=None).values, dtype=torch.double)

def gamma_logpdf_pytorch(x, alpha, beta):
    """Log pdf of the Gamma distribution."""    
    return (alpha*torch.log(beta) - torch.lgamma(alpha) + (alpha-1) * torch.log(x) - beta * x)



target = lambda theta: -gamma_logpdf_pytorch(data, torch.exp(theta[0]), torch.exp(theta[1])).sum()


theta = tensor([0., 0.], requires_grad=True)
optim = torch.optim.LBFGS([theta])


def closure():
    optim.zero_grad()
    nllh = target(theta)
    nllh.backward()
    return nllh

optim.step(closure)
a, b = torch.exp(theta.detach())
a,b

In [ ]:

x = torch.linspace(0, 10, 1000)
fig, ax = plt.subplots()
sns.histplot(data.cpu(), bins=50, stat='density', ax=ax)
sns.lineplot(x=x.cpu(), y=torch.exp(gamma_logpdf_pytorch(x, a, b).cpu()), ax=ax)

## 3. Student-$t$ regression (6 points)

In this task we will apply linear regression to data from the Framingham Heart Study that studies the association between heart disease and its causes.

A description of the data and its fields can be found at http://www.maths.utas.edu.au/DHStat/Data/Flow.html (see also problem 10.4 in the computer tasks done in class).

In this exercise, we will use the [Student-$t$ distribution](https://en.wikipedia.org/wiki/Student%27s_t-distribution) for the noise. This provides a form of robust regression due to the heavy tails of the Student-$t$ distribution, which makes it less affected by outliers.

Linear regression fit to scalar $x_i, y_i$ involves fitting the model
$$ y_i = \alpha + \beta x_i + \epsilon_i, $$
where $\beta$ is the regression coefficient and $\alpha$ is the intercept. Assuming $\epsilon_i \sim p(\epsilon_i | \sigma)$, the log-likelihood of the model is
$$ \log p(Y | X, \alpha, \beta, \sigma) = \sum_{i=1}^n \log p(y_i | x_i, \alpha, \beta, \sigma)
  = \sum_{i=1}^n \log p_{\epsilon_i}(y_i - \alpha - \beta x_i;\; \sigma), $$
where $p_{\epsilon_i}(y_i - \alpha - \beta x_i;\; \sigma)$ is the probability density of
the noise $\epsilon_i$, evaluated at $y_i - \alpha - \beta x_i$. 
In Problem 3 from Week 2, and in the computer task 10.3 and 10.4, we used a normal distribution for $p_{\epsilon_i}$ whereas here we will use the Student-$t$ distribution with $\nu = 5$ degrees of freedom and scale parameter $\sigma = 1$.

1. Load the data using the below code and plot the data.
2. Fit the data using standard linear regression with normally distributed errors ($p_{\epsilon_i} = \mathcal{N}(\epsilon_i;\; 0, \sigma^2)$ with $\sigma^2 = 1$). Report the $\alpha$ and $\beta$ you found in Moodle. (*Hint*: Problem 3 from Week 2 contains the formulas for $\alpha$ and $\beta$.)
3. Implement linear regression using the Student-$t$ log-likelihood function ($p_{\epsilon_i} = \text{Student-}t(\epsilon_i;\; 0, \sigma, \nu)$) with degrees of freedom (df) $\nu = 5$ and $\sigma = 1$, and maximise it (i.e. minimise the negative log-likelihood). Plot the fitted regression line in the same figure as the data.
4. Compare the coefficients $\alpha$ and $\beta$ you found for Student-$t$ regression with those from the standard linear regression. What can you observe? Report the coefficients you found in Moodle.

*Hint*: Feel free to use the Student-$t$ log-likelihood function implementations provided below.

$$ \hat{\beta}= \frac{\sum_{i = 1}^n (x_i - \bar{x})(y_i - \bar{y}) }{ \sum_{i = 1}^n (x_i - \bar{x})^2} \\
\hat{\alpha} = \bar{y} - \hat{\beta} \bar{x},$$
where $\bar{x} = \frac{1}{n} \sum_{i = 1}^n x_i$ and $\bar{y} = \frac{1}{n} \sum_{i = 1}^n y_i$.

In [ ]:
# Code for PyTorch
%matplotlib inline
import pandas as pd
import torch


@dataclass
class RegressionResult:
    alpha: Tensor
    beta: Tensor



# load the data from CSV file using pandas
fram = pd.read_csv('https://raw.githubusercontent.com/lacerbi/compstats-files/main/data/fram.txt', sep='\t')
# convert the variables of interest to numpy arrays
# input: Framingham relative weight - the ratio of the subjects weight to the median weight for their sex-height group
x = tensor(fram['CHOL'].values, dtype=torch.double)
# target: Systolic blood pressure, examination 1
y = tensor(fram['SBP'].values, dtype=torch.double)

x_cpu = x.cpu()
y_cpu = y.cpu()


# def student_logpdf_pytorch(x: Tensor, nu: Tensor) -> Tensor:
#     """Log pdf of Student-t distribution with nu degrees of freedom."""    
#     return torch.lgamma(0.5*(nu+1)) - torch.lgamma(0.5*nu) - 0.5*torch.log(tensor(torch.pi)) - 0.5*torch.log(nu) - 0.5*(nu+1)*torch.log(1 + x**2/nu)

# def linear_regression_normal(y: Tensor, x: Tensor) -> RegressionResult:
#     beta_hat = torch.sum((x-x.mean())*(y-y.mean())) / torch.sum((x-x.mean())**2)
#     alpha_hat = y.mean() - beta_hat * x.mean()
#     return RegressionResult(alpha_hat.cpu(), beta_hat.cpu())

In [ ]:
# normal_regression_result = linear_regression_normal(y, x)
# print(normal_regression_result)

In [ ]:


def student_t_regression_target(y: Tensor, x: Tensor, theta: Tensor, nu: Tensor
                                ) -> Tensor:
        
        return -torch.sum(torch.distributions.StudentT(df=nu, loc=0, scale=1).log_prob(y - theta[0] - (theta[1] * x)))

theta = tensor([0., 0.], requires_grad=True)



In [ ]:
optim = torch.optim.LBFGS([theta])
def solve_regression():
    optim.zero_grad()
    nllh = student_t_regression_target(y, x, theta, nu=tensor(5.))
    print(f"{nllh.item():.2f} -> {theta}")
    
    nllh.backward()
    return nllh

loss = optim.step(solve_regression)
print(loss)
print(theta)

In [ ]:
fig, ax = plt.subplots(1, 2)

sns.scatterplot(x=x_cpu, y=y_cpu, ax=ax[0], alpha=0.25)
sns.lineplot(x=x_cpu, y=131.73220676134514 + 0.06969627715093442*x_cpu, ax=ax[0])
ax[0].set_title(f'Linear regression: ($\\alpha={131.73220676134514:.2f}$, $\\beta={0.06969627715093442:.2f}$)')


sns.scatterplot(x=x_cpu, y=y_cpu, ax=ax[1], alpha=0.25)
sns.lineplot(x=x_cpu, y=8.8816 + 0.5359*x_cpu, ax=ax[1])
ax[1].set_title(f'Robust regression: ($\\nu=5$, $\\alpha={8.8816:.2f}$, $\\beta={ 0.5359:.2f}$)')


## 4. Variational inference for a mixture model (6 points)

In this problem, you are asked to estimate the posterior over the parameters of a mixture model using variational inference.

Find the posterior for the mixing weight $w$ and means $\mu_1, \mu_2$ in the normal mixture model
$$ p(x_i | \pi, \mu_1, \mu_2) = w \mathcal{N}(x_i;\; \mu_1, 1^2) + (1-w) \mathcal{N}(x_i;\; \mu_2, 1^2) $$
for the data set loaded below.

In order to ensure $0 \le w \le 1$, we will repar*ametrise it using the logistic transformation as $w = 1 / (1 + \exp(-\gamma))$. 

We will use normal priors for $\mu_1, \mu_2$ and a logistic-normal prior for $w$, which is equivalent to setting a normal prior for $\gamma$. The exact priors are
$$ p(\mu_1) = \mathcal{N}(\mu_1;\; 0, \sqrt{10}^2) \quad 
   p(\mu_2) = \mathcal{N}(\mu_2;\; 0, \sqrt{10}^2) \quad
   p(\gamma) = \mathcal{N}(\gamma;\; 0, 1.78^2), $$
and as usual we assume factorized (i.e., independent) priors, $p(\mu_1, \mu_2, \gamma) = p(\mu_1) p(\mu_2) p(\gamma)$.

As variational distribution we will use a product of independent distributions,

$$q(\mu_1, \mu_2, \gamma) = q(\mu_1) \cdot q(\mu_2) \cdot q(\gamma)$$

where  $q(\mu_1)$, $q(\mu_2)$ and $q(\gamma)$ are normal distributions. Fit the variational approximation $q(\mu_1, \mu_2, \gamma)$ to the posterior distribution of the model.

Use the stochastic gradient algorithm introduced in Chapter 12 of the course notes (doubly-stochastic variational inference,  DSVI). Please note that the independence assumption above is equivalent to saying that $q(\mu_1, \mu_2, \gamma)$ is a multivariate normal with diagonal covariance matrix $\Sigma$.

Report the means and standard deviations of $q(\mu_1)$ and $q(\mu_2)$, and the mean and standard deviation of $w$ under $q(\gamma)$ in Moodle. The required tolerance is $\pm 0.03$. When reporting the values to Moodle, please order the components so that the mean of the first component is lower than the mean of the second component, i.e. $\mathbb{E}[\mu_1] < \mathbb{E}[\mu_2]$.

*Hints*: 
- Computing the mean and standard deviation of $w$ requires extra effort because the approximation $q(\gamma)$ is over $\gamma$. The easiest way to solve this problem is to draw a number of samples from $q(\gamma)$, transform those to $w$ and compute the mean and standard deviation of the transformed samples.
- Note that the prior $p(\gamma)$ is already defined in the transformed (unconstrained) space, so you do not need to apply the Jacobian correction to the density used when transforming variables.
- You can reuse the code for DSVI provided in the class computer tasks, but note that it works for 1-D targets. You will need to adapt it to work with a 3-D target (or otherwise write your own code from scratch).
- Be sure that you report positive values for the standard deviation.

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

dataframe = pd.read_csv('https://raw.githubusercontent.com/lacerbi/compstats-files/main/data/mixture_data2.txt', header=None, sep='\t')
mixdata = tensor(dataframe.values[:,0])

In [4]:

# priors

def mu_1_prior(x: Tensor) -> Tensor:
    return torch.distributions.Normal(0, math.sqrt(10)).log_prob(x)

def mu_2_prior(x: Tensor) -> Tensor:
    return torch.distributions.Normal(0, math.sqrt(10)).log_prob(x)

def gamma_prior(x: Tensor) -> Tensor:
    return torch.distributions.Normal(0, .178).log_prob(x)

def w(gamma: Tensor) -> Tensor:
    return torch.sigmoid(gamma)

def w_prior(x: Tensor) -> Tensor:
    return gamma_prior(w(x))

def check_theta(theta: Tensor) -> None:
    assert theta.shape == (3,)
    if torch.isnan(theta).any():
        breakpoint()
        raise ValueError(f"NaN in theta {theta}")
    if torch.isinf(theta).any():
        breakpoint()
        raise ValueError(f"Inf in theta {theta}")

def factorised_log_prior(theta: Tensor) -> Tensor:
    return mu_1_prior(theta[0]) + mu_2_prior(theta[1]) + w_prior(theta[2])

def log_likelihood_mixture_model(theta: Tensor, x: Tensor) -> Tensor:
    
    log_mixture_1 = torch.log(w(theta[2])) + torch.distributions.Normal(theta[0], 1).log_prob(x)
    log_mixture_2 = torch.log(1-w(theta[2])) + torch.distributions.Normal(theta[1], 1).log_prob(x)
    return torch.logsumexp(torch.stack([log_mixture_1, log_mixture_2], dim=0), dim=0).sum()

def log_posterior(theta: Tensor, x: Tensor) -> Tensor:
    log_prior = factorised_log_prior(theta)
    log_lh = log_likelihood_mixture_model(theta, x)
    return (log_prior + log_lh)

def eta(shape: int = 3) -> Tensor:
    return torch.randn(shape)


def dsvi(m0: Tensor, c0: Tensor, n_iterations: int) -> tuple[Tensor, Tensor, Tensor, Tensor]:
    
    mu1_samples = torch.zeros(n_iterations)
    mu2_samples = torch.zeros(n_iterations)
    w_samples = torch.zeros(n_iterations)

    m = m0
    c = c0
    rho0 = 1e-1
    t0 = 100

    for t in range(n_iterations):
        
        # Re-parametrization trick
        eta_ = eta() + 1e-10
        theta: Tensor = (c @ eta_ + m).detach().requires_grad_(True)
        check_theta(theta)

        # Compute posterior and gradient
        posterior = log_posterior(theta, mixdata)
        posterior.backward()
        g = theta.grad
        
        # Update the parameters based on the gradient
        m = m + rho0 / (t + t0) * g
        c = c + rho0 / (t + t0) * (torch.outer(g, eta_.T) + (1/torch.diag(c)))


        # Store the samples
        theta.grad.zero_()
        mu1_samples[t] = theta[0]
        mu2_samples[t] = theta[1]
        w_samples[t] = theta[2]
        
    return mu1_samples, mu2_samples, w_samples

n_iterations = 25_000
n_trials = 20
mu1_samples = torch.zeros((n_trials, n_iterations))
mu2_samples = torch.zeros((n_trials, n_iterations))
gamma_samples = torch.zeros((n_trials, n_iterations))

for i in range(n_trials):
    m0 = torch.zeros(3)
    c0 = torch.eye(3)
    mu1_samples[i], mu2_samples[i], gamma_samples[i] = dsvi(m0, c0, n_iterations)

In [ ]:
# merge the mu_1 and mu2_ tensors so that we have a single tensor of shape (n_trials * 2, n_iterations,)
mu_samples = torch.cat([mu1_samples, mu2_samples], dim=0)

# split based on whether the value of mean(dim=0) is >= 0
upper_mu = mu_samples[mu_samples.mean(dim=1) >= 0, :]
lower_mu = mu_samples[mu_samples.mean(dim=1) < 0, :]

In [ ]:
print(upper_mu.mean(dim=1).std(), upper_mu.std(dim=1).std())
print(lower_mu.mean(dim=1).std(), lower_mu.std(dim=1).std())

In [ ]:
print(gamma_samples.mean(dim=1).std(), gamma_samples.std(dim=1).std())

In [ ]:
w_samples = torch.sigmoid(gamma_samples)
del(gamma_samples)

In [ ]:
print(w_samples.mean(dim=1).std(), w_samples.std(dim=1).std())

In [ ]:
# sns.set()

# mu1_cpu = mu1.detach().cpu()
# mu2_cpu = mu2.detach().cpu()
# gamma_cpu = gamma.detach().cpu()
# w_cpu = torch.sigmoid(gamma_cpu)

# fig, axes = plt.subplots(1, 3, figsize=(15, 5))
# fig.suptitle(f"Posterior samples (n={n_iterations}) for variational inference over a gaussian mixture model")
# sns.histplot(mu1_cpu, ax=axes[0])
# sns.histplot(mu2_cpu, ax=axes[1])
# sns.histplot(gamma_cpu, ax=axes[2])
# axes[0].set_title("$\mu_1$ - E[$\mu_1$] = {:.2f}".format(mu1_cpu.mean()))
# axes[1].set_title("$\mu_2$ - E[$\mu_2$] = {:.2f}".format(mu2_cpu.mean()))
# axes[2].set_title("$\gamma$ - E[$\gamma$] = {:.2f}".format(gamma_cpu.mean()))

In [ ]:
# fig, axes = plt.subplots(1, 2, figsize=(15, 5))
# fig.suptitle(f"Posterior samples of $\gamma$ and $w$ for the gaussian mixture model")
# sns.histplot(gamma_cpu, ax=axes[0])
# sns.histplot(w_cpu, ax=axes[1])
# axes[0].set_title("$\gamma$ - E[$\gamma$] = {:.2f}".format(gamma_cpu.mean()))
# axes[1].set_title("$w$ - E[$w$] = {:.2f}".format(w_cpu.mean()))

In [ ]:
# e_mu1 = mu1_cpu.mean()
# e_mu2 = mu2_cpu.mean()
# e_w = torch.sigmoid(gamma_cpu.mean())

In [ ]:
# # plot the mixture model over the data
# x = torch.linspace(-6, 6, 10000)
# y1 = torch.distributions.Normal(e_mu1, 1).log_prob(x) + torch.log(e_w)
# y2 = torch.distributions.Normal(e_mu2, 1).log_prob(x) + torch.log(1-e_w)
# y = torch.logsumexp(torch.stack([y1, y2], dim=0), dim=0)

# fig, ax = plt.subplots()
# sns.histplot(mixdata.cpu(), bins=50, stat='density', ax=ax)
# sns.lineplot(x=x.cpu(), y=torch.exp(y.cpu()), ax=ax, color='red')
# sns.lineplot(x=x.cpu(), y=torch.exp(y1.cpu()), ax=ax, color='darkgreen', linestyle='--')
# sns.lineplot(x=x.cpu(), y=torch.exp(y2.cpu()), ax=ax, color='darkblue', linestyle='--')
# plt.axvline(e_mu1, color='darkgreen', linestyle='--')
# plt.axvline(e_mu2, color='darkblue', linestyle='--')